## Data Wrangling

In [1]:
#load python packages
import os
import numpy as np
import pandas as pd
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import stem

In [2]:
os.getcwd()

'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two'

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'Capstone Two',
 'data',
 'embeddings',
 'figures',
 'models',
 'Quora Insincere Capstone Data Wrangling.ipynb',
 'sample_submission.csv',
 'test.csv',
 'train.csv']

In [4]:
#path=""
#os.chdir(path)

### Data Collection 

#### Load train and test csv files

In [5]:
file_name_1 = 'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two\\train.csv'
file_name_2 = 'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two\\test.csv'

In [6]:
# Create train and test dataframes by reading the csv files
train = pd.read_csv(file_name_1)
test = pd.read_csv(file_name_2)

In [7]:
#Concat both train and test files for initial pre-processing
quora_df = pd.concat([train, test])

In [8]:
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0


In [9]:
#Number of texts retrieved from merged dataframe
quora_df.shape[0]

1424381

### Data Organization

In [10]:
path = 'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two'

In [11]:
print ("The current working directory is %s" % path)

The current working directory is C:\Shalin\Springboard bootcamp\projects\Capstone project 2\Capstone Two


#### Create subfolders in your project directory

In [14]:
data = os.makedirs('data')

In [15]:
figures = os.makedirs('figures')

In [16]:
models = os.makedirs('models')

In [12]:
os.listdir()

['.ipynb_checkpoints',
 'Capstone Two',
 'data',
 'embeddings',
 'figures',
 'models',
 'Quora Insincere Capstone Data Wrangling.ipynb',
 'sample_submission.csv',
 'test.csv',
 'train.csv']

### Data Definition

In [13]:
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1424381 entries, 0 to 375805
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   qid            1424381 non-null  object 
 1   question_text  1424381 non-null  object 
 2   target         1048575 non-null  float64
dtypes: float64(1), object(2)
memory usage: 43.5+ MB


In [14]:
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0


In [15]:
quora_df.tail(2)

,qid,question_text,target
375804,ffffb1f7f1a008620287,What are the causes of refraction of light?,NaN
375805,fffff85473f4699474b0,Climate change is a worrying topic. How much t...,NaN


In [16]:
quora_df.columns

Index(['qid', 'question_text', 'target'], dtype='object')

In [17]:
quora_df.dtypes

qid               object
question_text     object
target           float64
dtype: object

#### Data cleaning for target column

In [18]:
quora_df['target'].isna().sum()

375806

In [19]:
quora_df = quora_df[quora_df['target'].notna()]
quora_df.tail(2)

,qid,question_text,target
1048573,cd763d37d3d7cfcd42ce,What is the difference between the old currenc...,0.0
1048574,cd7642554d107f946d8a,What is the full form of DML?,0.0


In [20]:
quora_df['target'].isna().sum()

0

#### Convert target column into 'integer' since the values will either be 0 or 1 (prediction for sensitive vs insensitive data)

In [21]:
quora_df['target'] = quora_df['target'].apply(np.int64)

In [22]:
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   qid            1048575 non-null  object
 1   question_text  1048575 non-null  object
 2   target         1048575 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 32.0+ MB


In [23]:
quora_df.describe()

,target
count,1.048575e+06
mean,6.177336e-02
std,2.407436e-01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,1.000000e+00


**Extract only the question_text column from the dataframe to perform further data cleaning**

In [24]:
quora_question_df = quora_df['question_text']
quora_question_df.head(2)

0    How did Quebec nationalists see their province...
1    Do you have an adopted dog, how would you enco...
Name: question_text, dtype: object

**Find max and min length of question from the list**

In [25]:
max_len = 0
for length in quora_question_df: 
    if len(length) > max_len: 
        max_len = len(length) 
        question = length 
print("Maximum length string is : " + question) 

Maximum length string is : What is [math]\frac{\int_{1x^5}^{3x^{-5}} \tan(\tan({\boxed{\int_{1x^0}^{1x^2} \sum_{\varpi=1}^{\infty} \int_{2x^{-3}}^{2x^2} \sum_{\alpha=7}^{\infty} \underbrace{\sqrt[2]{1x^5}}_{\text{Gauss's Law of Theoretical Probability.}} d\tau dx}}^{1x^0})) d\mu}{\int_{2x^{-3}}^{1x^5} \cos(\int_{2x^2}^{1x^{-3}} \frac{\sqrt[2]{\overbrace{\underbrace{\frac{3x^3+3x^5}{\sqrt[3]{2x^{-3}}}}_{\text{Gauss's Law of Theoretical Probability.}} \times \overbrace{\tan(2x^0)}^{\text{Gauss's Law of Theoretical Probability.}}-\sum_{4=7}^{\infty} \boxed{3x^{-5}}}^{\text{Inverse Function.}}}}{{\boxed{\int_{2x^2}^{2x^4} 3x^1 d9} \div \sum_{6=6}^{\infty} \sqrt[3]{2x^2}+\sqrt[4]{\sin(2x^0+3x^0)}}^{2x^{-4}}+\boxed{\frac{\vec{\boxed{\sum_{\gamma=10}^{\infty} 1x^{-5}}}}{\frac{\sum_{\iota=2}^{\infty} 1x^{-5}-\frac{3x^{-1}}{1x^{-4}}}{\sin(\tan(3x^{-2}))}}}} \times \boxed{\sqrt[2]{{{{\sqrt[5]{2x^5}}^{2x^{-1}}}^{2x^{-1}} \div \sum_{\chi=6}^{\infty} \int_{1x^4}^{2x^{-4}} 3x^2 d\vartheta+{2x^{-3}}^

**Alternative way of finding the max length of question from the list**

In [26]:
question_max = max(quora_question_df, key = len) 
question_max

"What is [math]\\frac{\\int_{1x^5}^{3x^{-5}} \\tan(\\tan({\\boxed{\\int_{1x^0}^{1x^2} \\sum_{\\varpi=1}^{\\infty} \\int_{2x^{-3}}^{2x^2} \\sum_{\\alpha=7}^{\\infty} \\underbrace{\\sqrt[2]{1x^5}}_{\\text{Gauss's Law of Theoretical Probability.}} d\\tau dx}}^{1x^0})) d\\mu}{\\int_{2x^{-3}}^{1x^5} \\cos(\\int_{2x^2}^{1x^{-3}} \\frac{\\sqrt[2]{\\overbrace{\\underbrace{\\frac{3x^3+3x^5}{\\sqrt[3]{2x^{-3}}}}_{\\text{Gauss's Law of Theoretical Probability.}} \\times \\overbrace{\\tan(2x^0)}^{\\text{Gauss's Law of Theoretical Probability.}}-\\sum_{4=7}^{\\infty} \\boxed{3x^{-5}}}^{\\text{Inverse Function.}}}}{{\\boxed{\\int_{2x^2}^{2x^4} 3x^1 d9} \\div \\sum_{6=6}^{\\infty} \\sqrt[3]{2x^2}+\\sqrt[4]{\\sin(2x^0+3x^0)}}^{2x^{-4}}+\\boxed{\\frac{\\vec{\\boxed{\\sum_{\\gamma=10}^{\\infty} 1x^{-5}}}}{\\frac{\\sum_{\\iota=2}^{\\infty} 1x^{-5}-\\frac{3x^{-1}}{1x^{-4}}}{\\sin(\\tan(3x^{-2}))}}}} \\times \\boxed{\\sqrt[2]{{{{\\sqrt[5]{2x^5}}^{2x^{-1}}}^{2x^{-1}} \\div \\sum_{\\chi=6}^{\\infty} \\int_{1

In [27]:
question_min = min(quora_question_df, key = len) 
question_min

'"'

### Data Cleaning

As identified in earlier step during data definition step, null values were already removed from 'target' column of the dataframe. This data cleaning will be performed only on the 'question_text' column of dataframe

**Convert text to lowercase**

In [28]:
quora_df['question_text'] = quora_df['question_text'].str.lower()
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,how did quebec nationalists see their province...,0
1,000032939017120e6e44,"do you have an adopted dog, how would you enco...",0


**Remove punctuation**
The following code removes this set of symbols [!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]:

In [29]:
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    return s

quora_df['question_text'] = quora_df['question_text'].apply(remove_punctuation)

In [30]:
quora_question_df = quora_df['question_text']
quora_question_df.head(2)

0    how did quebec nationalists see their province...
1    do you have an adopted dog how would you encou...
Name: question_text, dtype: object

There are whitespaces, new line characters which can be removed (on deriving length and min length strings)

In [31]:
question_max = max(quora_question_df, key = len) 
question_max

'in star trek 2013 why did they \n\nspoilers\nspoilers\nspoilers\nspoilers\n\n1make warping look quite a bit like an hyperspace jump\n2what in the world were those bright particles as soon as they jumped\n3why in the world did they make it possible for two entities to react in warp space in separate jumps\n4why did spock get emotions for this movie\n5what was the point of hiding the enterprise underwater\n6when they were intercepted by the dark ship how come they reached earth when they were far away from heri dont seem to remember the scene where they warp to earth\n7how did the ship enter earths atmosphere when it wasnt even in orbit\n8when scotty opened the door of the black ship  how come pike and khan didnt slow down'

In [32]:
question_min = min(quora_question_df, key = len) 
question_min

''

**Remove newline characters and whitespaces**

- Remove newline characters

In [33]:
quora_df['question_text']= [re.sub(r'\n\d', '?', text).replace("\nspoilers","").replace("\n","") for text in quora_df['question_text']]
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,how did quebec nationalists see their province...,0
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0


In [34]:
quora_question_df = quora_df['question_text']
quora_question_df.head(2)

0    how did quebec nationalists see their province...
1    do you have an adopted dog how would you encou...
Name: question_text, dtype: object

In [35]:
question_max = max(quora_question_df, key = len) 
question_max

'in star trek 2013 why did they ?make warping look quite a bit like an hyperspace jump?what in the world were those bright particles as soon as they jumped?why in the world did they make it possible for two entities to react in warp space in separate jumps?why did spock get emotions for this movie?what was the point of hiding the enterprise underwater?when they were intercepted by the dark ship how come they reached earth when they were far away from heri dont seem to remember the scene where they warp to earth?how did the ship enter earths atmosphere when it wasnt even in orbit?when scotty opened the door of the black ship  how come pike and khan didnt slow down'

- Remove whitespaces : to remove leading and ending spaces, you can use the strip() function:

In [36]:
quora_df['question_text'] = [x.strip() for x in quora_df['question_text']]
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,how did quebec nationalists see their province...,0
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0


#### Handling missing values in case of empty string values

In [37]:
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   qid            1048575 non-null  object
 1   question_text  1048575 non-null  object
 2   target         1048575 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 32.0+ MB


In [38]:
# Find out if there are any missing question_text values
null_df = [text for text in quora_df['question_text'] if len(text) < 1]
null_df

['']

In [39]:
null_df = np.where(quora_df['question_text'] == '')
null_df

(array([420816], dtype=int64),)

In [40]:
quora_df.iloc[420816]

qid              527aac2ce6f12f789fe5
question_text                        
target                              1
Name: 420816, dtype: object

In [41]:
quora_df = quora_df.drop(quora_df.index[420816])
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048574 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   qid            1048574 non-null  object
 1   question_text  1048574 non-null  object
 2   target         1048574 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 32.0+ MB


**Remove stop words**
- “Stop words” are the most common words in a language like “the”, “a”, “on”, “is”, “all”. These words do not carry important meaning and are usually removed from texts. It is possible to remove stop words using Natural Language Toolkit (NLTK), a suite of libraries and programs for symbolic and statistical natural language processing.

In [42]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gosal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gosal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,how did quebec nationalists see their province...,0
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0


In [44]:
stop_words = stopwords.words('english')

quora_df['questions_without_stopwords'] = quora_df['question_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
quora_df.head(2)

,qid,question_text,target,questions_without_stopwords
0,00002165364db923c7e6,how did quebec nationalists see their province...,0,quebec nationalists see province nation 1960s
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0,adopted dog would encourage people adopt shop


In [45]:
del quora_df['question_text']

In [46]:
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048574 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   qid                          1048574 non-null  object
 1   target                       1048574 non-null  int64 
 2   questions_without_stopwords  1048574 non-null  object
dtypes: int64(1), object(2)
memory usage: 32.0+ MB


**Stemming using NLTK**
- Stemming is a process of reducing words to their word stem, base or root form (for example, books — book, looked — look). The main two algorithms are Porter stemming algorithm (removes common morphological and inflexional endings from words [14]) and Lancaster stemming algorithm (a more aggressive stemming algorithm). In the “Stemming” sheet of the table some stemmers are described

In [47]:
stemmer=PorterStemmer()

quora_df['question_stemmed'] = quora_df['questions_without_stopwords'].apply(lambda x: stemmer.stem(x)) # Stem every word.
quora_df = quora_df.drop(columns=['questions_without_stopwords']) # Get rid of the unstemmed column.
quora_df.head(2) # Print dataframe.

,qid,target,question_stemmed
0,00002165364db923c7e6,0,quebec nationalists see province nation 1960
1,000032939017120e6e44,0,adopted dog would encourage people adopt shop


#### Add columns to include a) length of each question b) # of words in the dataframe (for any future exercise)

In [48]:
quora_df['question_length']  = quora_df['question_stemmed'].str.len()
quora_df['total_words'] = [len(x.split()) for x in quora_df['question_stemmed'].tolist()]
quora_df.head(2)

,qid,target,question_stemmed,question_length,total_words
0,00002165364db923c7e6,0,quebec nationalists see province nation 1960,44,6
1,000032939017120e6e44,0,adopted dog would encourage people adopt shop,45,7


In [49]:
quora_df.sort_values(by='question_length', ascending=False, inplace=True)
quora_df = quora_df[['qid', 'question_stemmed', 'target', 'question_length', 'total_words']]
quora_df.head(10)

,qid,question_stemmed,target,question_length,total_words
443216,56da6b6875d686b48fde,mathfracint1x53x5 tantanboxedint1x01x2 sumvarp...,1,589,40
163583,1ffca149bd0a19cd714c,mathoverbracesumvartheta8infty vecfracsumkappa...,1,498,31
522266,663c7523d48f5ee66a3e,star trek 2013 ?make warping look quite bit li...,0,449,66
341312,42dc38e62f0bdebfa6b6,answer mathfrac4222262mid 5501int846fracomega2...,0,334,11
149952,1d53c9c017999b4f77e2,8430397824532987451912384179815150754023741609...,0,241,3
393959,4d2e2796dd1ced2c8e64,prospects international students httpwwwophamm...,0,240,12
118298,1729272298ed7f6cf789,could probing quantum world vacuum virtual par...,0,221,26
1004888,c4ee0b28e195a3d515a5,mention good book containing quotations great ...,0,219,29
460851,5a3f6882c1fc8319eff1,didnt genghiz khan thee great khan abolish des...,1,218,32
543322,6a71d18ec595137eb634,india harbor many scammers thieves con artists...,1,218,27


### Data cleaning continued

#### Removing entries that have 0 'question length'

In [50]:
quora_df = quora_df[quora_df['question_length'] != 0]
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048523 entries, 443216 to 770283
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   qid               1048523 non-null  object
 1   question_stemmed  1048523 non-null  object
 2   target            1048523 non-null  int64 
 3   question_length   1048523 non-null  int64 
 4   total_words       1048523 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 48.0+ MB


As you could see above, more than 50 rows were removed since there was no text associated to them. In an earlier cleaning step, there was an effort to remove entries with missing values with text length < 1, but that didnt remove everything (maybe those were whitespaces yet didnt account for len(text)<1)

In [51]:
quora_df.head(5)

,qid,question_stemmed,target,question_length,total_words
443216,56da6b6875d686b48fde,mathfracint1x53x5 tantanboxedint1x01x2 sumvarp...,1,589,40
163583,1ffca149bd0a19cd714c,mathoverbracesumvartheta8infty vecfracsumkappa...,1,498,31
522266,663c7523d48f5ee66a3e,star trek 2013 ?make warping look quite bit li...,0,449,66
341312,42dc38e62f0bdebfa6b6,answer mathfrac4222262mid 5501int846fracomega2...,0,334,11
149952,1d53c9c017999b4f77e2,8430397824532987451912384179815150754023741609...,0,241,3


#### Removing rows with very long words (likely junk values) 
- As seen above, there are still many records that have a very long character length and can be removed (we can choose 15 characters)
- Also alphanumeric values (that have mix of numbers of strings) can be filtered out
- However, need to be wary of the fact that since punctuations were removed, we may have certain rows with http OR www values AND could have a mix of numbers and characters AND still could exceed the threshold word length (as described in the above point) that we are looking to filter, hence they need to be retained 

In [52]:
quora_df = quora_df[~quora_df['question_stemmed'].str.contains(r'(?<!\S)(?!http|\S*www\.)\S{15}')]

In [53]:
quora_df.head(5)

,qid,question_stemmed,target,question_length,total_words
393959,4d2e2796dd1ced2c8e64,prospects international students httpwwwophamm...,0,240,12
118298,1729272298ed7f6cf789,could probing quantum world vacuum virtual par...,0,221,26
1004888,c4ee0b28e195a3d515a5,mention good book containing quotations great ...,0,219,29
460851,5a3f6882c1fc8319eff1,didnt genghiz khan thee great khan abolish des...,1,218,32
543322,6a71d18ec595137eb634,india harbor many scammers thieves con artists...,1,218,27


In [54]:
quora_df.tail(5)

,qid,question_stemmed,target,question_length,total_words
302549,3b3f408e3f9a1ea97fb7,n,0,1,1
995257,c309469a202434b5f1d2,w,1,1,1
894300,af37c56bf566734dc780,c,0,1,1
69097,0d8c9f2ac8285c2e5070,b,0,1,1
770283,96eab2e1b7dfeed0546d,i,0,1,1


- Find count of all question_text grouping to see the question_length

In [55]:
question_length_grouping = quora_df.groupby('question_length').question_stemmed.nunique().head(15)
question_length_grouping

question_length
1        10
2        87
3       343
4       711
5       840
6       983
7      1526
8      2705
9      3897
10     4848
11     5872
12     6897
13     8401
14     9812
15    11566
Name: question_stemmed, dtype: int64

In [56]:
char_len_9 = quora_df[quora_df['question_length'] == 9]
char_len_9.tail(5)

,qid,question_stemmed,target,question_length,total_words
314283,3d96c60f9994349058d4,wrong cat,0,9,2
304920,3bb995601ca3d2b5556a,mathcircl,0,9,1
616568,78bdf2af0fa0275cbafd,planets s,0,9,2
998573,c3aebcff7dd7561d2fcd,step loop,0,9,2
47052,0935bdf95841f149832b,good plac,0,9,2


In [57]:
char_len_10 = quora_df[quora_df['question_length'] == 10]
char_len_10.tail(5)

,qid,question_stemmed,target,question_length,total_words
771205,9715ad3038d2ed0398a1,parts skul,0,10,2
344350,4378b8e3f8576a723bd4,harisarman,0,10,1
771192,9714e30829b60ec8e4bd,kinder oth,0,10,2
401447,4ea7cb1aa86e70e0cbc0,women evil,1,10,2
590808,73bafbe21255a162e7c7,people fal,0,10,2


#### Remove rows where length of text is < 10
- In earlier section, we removed all null text rows based on 'len' method
- However based on the above grouping, it seems like it might be worth to remove all rows where question length is <10. And also worth to remove rows that have just 1 word 

In [58]:
quora_df = quora_df[quora_df['question_length'] >= 10]
quora_df.head(5)

,qid,question_stemmed,target,question_length,total_words
393959,4d2e2796dd1ced2c8e64,prospects international students httpwwwophamm...,0,240,12
118298,1729272298ed7f6cf789,could probing quantum world vacuum virtual par...,0,221,26
1004888,c4ee0b28e195a3d515a5,mention good book containing quotations great ...,0,219,29
460851,5a3f6882c1fc8319eff1,didnt genghiz khan thee great khan abolish des...,1,218,32
543322,6a71d18ec595137eb634,india harbor many scammers thieves con artists...,1,218,27


In [61]:
quora_df = quora_df[quora_df['total_words'] > 1]
quora_df.head(5)

,qid,question_stemmed,target,question_length,total_words
393959,4d2e2796dd1ced2c8e64,prospects international students httpwwwophamm...,0,240,12
118298,1729272298ed7f6cf789,could probing quantum world vacuum virtual par...,0,221,26
1004888,c4ee0b28e195a3d515a5,mention good book containing quotations great ...,0,219,29
460851,5a3f6882c1fc8319eff1,didnt genghiz khan thee great khan abolish des...,1,218,32
543322,6a71d18ec595137eb634,india harbor many scammers thieves con artists...,1,218,27


In [63]:
quora_df.tail(5)

,qid,question_stemmed,target,question_length,total_words
946040,b963234a1b54de97badf,space ship,0,10,2
771205,9715ad3038d2ed0398a1,parts skul,0,10,2
771192,9714e30829b60ec8e4bd,kinder oth,0,10,2
401447,4ea7cb1aa86e70e0cbc0,women evil,1,10,2
590808,73bafbe21255a162e7c7,people fal,0,10,2


In [62]:
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1016694 entries, 393959 to 590808
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   qid               1016694 non-null  object
 1   question_stemmed  1016694 non-null  object
 2   target            1016694 non-null  int64 
 3   question_length   1016694 non-null  int64 
 4   total_words       1016694 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 46.5+ MB
